In [12]:
import numpy as np
import pandas as pd
import itertools as it
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.metrics import roc_curve, auc,precision_recall_curve, roc_curve
import matplotlib.pylab as plt

In [13]:
Data = pd.read_excel('Data.xlsx')
Data = Data.set_index('Date')
Y = Data['Class']
X = Data.drop('Class', axis = 1)

### Parameters

In [14]:
step = 100
ts_prop = 0.1
va_prop = 0.2

Hyper_Parameters = [[i for i in range(1,5)], # max_depth
                    [i for i in range(1,10)]] # min_samples_leaf
set_parameters = [i for i in it.product(*Hyper_Parameters)]
k = 5

#### Error real y de entrenamiento deseados

In [15]:
Epsilon = 0.15
delta = 0.15

#### Garantia probable de aprendizaje ($\eta_{estimado}$) y Tamaño optimo de la muestra $\eta_{optimo}$

#### $\eta_{estimado}$

In [16]:
n_est = int(np.ceil((1/Epsilon)*(np.log(len(set_parameters)) + np.log(1/delta))))

#### $\eta_{optimo}$

In [17]:
n_opt = int((np.log(2)/(2*Epsilon**2))*((2**k - 1)*(1 + np.log2(X.shape[1])) + 1 + np.log(delta**(-1))))

#### Division del conjunto: entrenamiento (70%), validacion (20%) y prueba (10%)

In [18]:
X1, X1_ts, Y1,Y1_ts = train_test_split(X, Y, train_size = 1-ts_prop, random_state=19)
X1_tr, X1_va, Y1_tr, Y1_va = train_test_split(X1, Y1, train_size = 1-va_prop, random_state=19)

#### Discretizacion para explorar el aumento de $\eta$

In [19]:
N = range(n_est, round(X1_tr.shape[0]*(1-va_prop)), step)

### Exploracion de modelos partiendo de $\eta_{estimado}$, es claro que $\eta_{estimado}$ es tenido en cuenta ya que $\eta_{estimado} < \eta_{optimo}$

In [20]:
n_est_mod_results = {
    'Model': [None for i in range(len(set_parameters))],
    'Acc_Test': np.zeros((len(N),len(set_parameters))),
    'Acc_Val': np.zeros((len(N),len(set_parameters)))
}

In [21]:
for j in range(len(set_parameters)):
    dtc = DecisionTreeClassifier(max_depth = set_parameters[j][0], min_samples_leaf= set_parameters[j][1], random_state=19)
    for i in range(len(N)):
        np.random.seed(seed = 19)
        ind1 = np.random.randint(0, X1_tr.shape[0], N[i])
        dtc.fit(X1_tr.iloc[ind1,], Y1_tr[ind1])
        y_pred = dtc.predict(X1_va)
        n_est_mod_results['Acc_Val'][i,j] = accuracy_score(Y1_va, y_pred)
    n_est_mod_results['Model'][j] = dtc
    
ind_bst = np.where(n_est_mod_results['Acc_Val'] == np.amax(n_est_mod_results['Acc_Val']))[1][0]
n_est_mod_results['Best'] = set_parameters[ind_bst]

#### Grid search

In [22]:
Hyper_Parameters = {
    'max_depth': [i for i in range(1,5)],
    'min_samples_leaf': [i for i in range(1,10)]
}
clf = GridSearchCV(DecisionTreeClassifier(), Hyper_Parameters, cv = 5)
clf.fit(X1_tr.values, Y1_tr)
clf.best_params_

{'max_depth': 4, 'min_samples_leaf': 1}